In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
#Nacitanie dat
data = pd.read_csv("data.csv", sep = ",")

/var/folders/_v/69trf4797bjdm6rrmtk051240000gn/T/ipykernel_95135/3681040214.py:2: DtypeWarning: Columns (5,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data.csv", sep = ",")


In [ ]:
data.head(5)

PRETYPOVANIE ATRIBÚTOV + ODSTRÁNENIE CHÝBAJÚCICH HODNÔT

In [ ]:
data.dtypes

In [3]:
#Pretypovanie atributov datum a cas na format datetime
data['datum'] = pd.to_datetime(data['datum'], format='mixed')
data['cas'] = pd.to_datetime(data['cas'])

/var/folders/_v/69trf4797bjdm6rrmtk051240000gn/T/ipykernel_95135/3825292402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['cas'] = pd.to_datetime(data['cas'])


In [ ]:
#Rozdelenie atributu datum do 3 atributov (den, mesiac, rok)
data['rok'] = data['datum'].apply(lambda time: time.year)
data['mesiac'] = data['datum'].apply(lambda time: time.month)
data['den'] = data['datum'].apply(lambda time: time.day)
data = data.drop(['datum'], axis = 1)

#Uprava atributu cas na cele hodiny
data['cas'] = data['cas'].apply(lambda time: time.hour)

In [ ]:
#Vymazanie atributov s vysokym poctom chybajucich hodnot
data = data.drop(['latitude', 'longitude', 'regional', 'delegacia', 'uop'], axis=1)

In [ ]:
#Zmena poradia atributov
data = data[['den', 'mesiac', 'rok', 'den_v_tyzdni', 'cas', 'stat', 'cislo_dialnice', 'kilometer', 'mesto', 'pricina_nehody', 'druh_nehody', 'faza_dna', 'pocasie', 'sklon_vozovky', 'typ_dialnice', 'miesto_nehody', 'lokacia_dialnice', 'pocet_vozidiel', 'pocet_osob', 'celkovy_pocet_zranenych', 'pocet_mrtvych', 'pocet_lahko_zranenych', 'pocet_tazko_zranenych', 'pocet_nezranenych', 'pocet_neuvedenych', 'klasifikacia_nehody']]

In [ ]:
#Pocet chybajucich hodnot pre kazdy atribut
data.isna().sum()

In [ ]:
#Odstranenie zaznamov s prazdnymi hodnotami
data = data.dropna(subset=['kilometer', 'druh_nehody', 'faza_dna', 'pricina_nehody', 'pocasie', 'typ_dialnice', 'miesto_nehody', 'lokacia_dialnice'])

In [ ]:
#Pretypovanie atributu cislo_dialnice na object
data['cislo_dialnice'] = data['cislo_dialnice'].astype(int).astype(str)

In [ ]:
#Nahradenie desatinnej ciarky bodkou pre konzistentnost atributu kilometer
def convert_to_float_or_str(value):
    try:
        if isinstance(value, str):
            return float(value.replace(',', '.'))
        else:
            return value
    except ValueError:
        return None

data['kilometer'] = data['kilometer'].apply(convert_to_float_or_str)

In [ ]:
#Pocet prikladov pre kazdu unikatnu hodnotu atributu klasifikacia nehody
data['klasifikacia_nehody'].value_counts()

In [ ]:
# Upravenie atributu klasifikacia nehody na zaklade novych kriterii
def uprav_klasifikaciu(riadok):
    if riadok['pocet_mrtvych'] >= 1:
        return 'Smrtelna nehoda'
    elif riadok['pocet_mrtvych'] == 0 and riadok['pocet_lahko_zranenych'] == 0 and riadok['pocet_tazko_zranenych'] == 0 and riadok['pocet_nezranenych'] >= 1:
        return 'Nehoda bez zraneni'
    elif riadok['pocet_tazko_zranenych'] >= 1:
        return 'Nehoda s tazkymi zraneniami'
    else:
        return 'Nehoda s lahkymi zraneniami'

data['klasifikacia_nehody'] = data.apply(uprav_klasifikaciu, axis=1)

Vytvorenie cieľového atribútu

In [ ]:
#Vytvorenie cieloveho atributu nebezpecenstvo
def urcenie_nebezpecenstva(riadok):
    if riadok['klasifikacia_nehody'] in ['Smrtelna nehoda', 'Nehoda s tazkymi zraneniami']:
        return 1
    else:
        return 0

data['nebezpecenstvo'] = data.apply(urcenie_nebezpecenstva, axis=1)

In [ ]:
#Pocet unikatnych hodnot atributu nebezpecenstvo
data['nebezpecenstvo'].value_counts()

In [ ]:
#Funkcia na urcenie bezpecnych a nebezpecnych dialnic
def aktualizuj_nebezpecnost(data):
    #Zoznam dialnic
    unikatne_dialnice = data['cislo_dialnice'].unique()
    
    for dialnica in unikatne_dialnice:
        pocet_nebezpecnych = data[(data['cislo_dialnice'] == dialnica) & (data['nebezpecenstvo'] == 1)].shape[0]
        pocet_bezpecnych = data[(data['cislo_dialnice'] == dialnica) & (data['nebezpecenstvo'] == 0)].shape[0]
        percento_nebezpecnych = (pocet_nebezpecnych / (pocet_nebezpecnych+pocet_bezpecnych))
        
        #Ak doslo k max 15 nehodam, dialnica je bezpecna
        if pocet_nebezpecnych + pocet_bezpecnych <= 15:
            data.loc[data['cislo_dialnice'] == dialnica, 'nebezpecenstvo'] = 0
        #Ak 15% a viac nehod na danej dialnici je nebezpecnych, dialnica je nebezpecna
        elif percento_nebezpecnych >= 0.15:
            data.loc[data['cislo_dialnice'] == dialnica, 'nebezpecenstvo'] = 1
        #Ostatne dialnice su bezpecne
        else:
            data.loc[data['cislo_dialnice'] == dialnica, 'nebezpecenstvo'] = 0
    
    return data

data = aktualizuj_nebezpecnost(data)

In [ ]:
#Pocet unikatnych hodnot atributu nebezpecenstvo
data['nebezpecenstvo'].value_counts()

In [ ]:
#Pocet bezpecnych a nebezpecnych dialnic
pocet_dialnic_s_0 = data[data['nebezpecenstvo'] == 0]['cislo_dialnice'].nunique()
pocet_dialnic_s_1 = data[data['nebezpecenstvo'] == 1]['cislo_dialnice'].nunique()

print(f"Počet dialníc s hodnotou nebezpečenstva 0: {pocet_dialnic_s_0}")
print(f"Počet dialníc s hodnotou nebezpečenstva 1: {pocet_dialnic_s_1}")

KORELÁCIA MEDZI NUMERICKÝMI ATRIBÚTMI

In [ ]:
num_data = data[['den', 'cas', 'kilometer', 'pocet_osob', 'pocet_neuvedenych', 'pocet_vozidiel', 'celkovy_pocet_zranenych', 'pocet_mrtvych', 'pocet_lahko_zranenych', 'pocet_tazko_zranenych', 'pocet_nezranenych']]
corr_matrix = num_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
#Odstranie atributov, ktore boli pouzite pri uprave inych atributov, taktiez atributy s vysokou a strednou korelaciou
data = data.drop(['cislo_dialnice','pocet_mrtvych', 'pocet_lahko_zranenych', 'pocet_tazko_zranenych', 'pocet_nezranenych', 'klasifikacia_nehody', 'pocet_osob'], axis=1)

Konverzia kategorických atribútov na numerické

In [ ]:
#Zmena kategorickych atributov na numericke
enc = LabelEncoder()
data["den_v_tyzdni"] = enc.fit_transform(data["den_v_tyzdni"])
data["faza_dna"] = enc.fit_transform(data["faza_dna"])
data["sklon_vozovky"] = enc.fit_transform(data["sklon_vozovky"])
data["pocasie"] = enc.fit_transform(data["pocasie"])
data["typ_dialnice"] = enc.fit_transform(data["typ_dialnice"])
data["miesto_nehody"] = enc.fit_transform(data["miesto_nehody"])
data["lokacia_dialnice"] = enc.fit_transform(data["lokacia_dialnice"])
data["stat"] = enc.fit_transform(data["stat"])
data["mesto"] = enc.fit_transform(data["mesto"])
data["pricina_nehody"] = enc.fit_transform(data["pricina_nehody"])
data["druh_nehody"] = enc.fit_transform(data["druh_nehody"])

Kroková regresia

In [ ]:
#Vyber vzorky dat
data_sample = data.sample(n=50000, random_state=8391)

#Funkcia krokovej regresie
def selection_auc(data_sample, target_column, forward, type):
    X = data_sample.drop(columns=[target_column])
    y = data_sample[target_column]
    feature_selector = SequentialFeatureSelector(LogisticRegression(max_iter = 1000000),
                                                 k_features='best',
                                                 forward=forward,
                                                 scoring='roc_auc')
    feature_selector.fit(X, y)
    selected_features = list(X.columns[list(feature_selector.k_feature_idx_)])
    print(f"{type}: {selected_features}")
    return selected_features

In [ ]:
# Vypis atributov oboma krokovymi regresiami
selected_features_forward_auc = selection_auc(data_sample, 'nebezpecenstvo', True, "FORWARD")
selected_features_backward_auc = selection_auc(data_sample, 'nebezpecenstvo', False, "BACKWARD")

In [ ]:
#Ulozenie predspracovanych dat
data.to_csv('data.csv', index=False)